In [1]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)


Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("./나무문화/2022/나무문화_본문_2022_9월.csv", encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [3]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [4]:
df

,url,본문,org_idx,cleaning
0,https://blog.naver.com/plumtreelee-/222888764192,#2022정조대왕능행차공동재현 #정조대왕능행차\n2022 정조대왕 능행차 공동재현\...,0,2022정조대왕능행차공동재현 정조대왕능행차 2022 정조대왕 능행차 공동재현 Kin...
1,https://blog.naver.com/kimjihye525/222888763332,연꽃이 필때 와보고싶었지만\n더위에 약한 나는 이제서야 ...\n경기장튀김덕에 그래...,1,연꽃이 필때 와보고싶었지만 더위에 약한 나는 이제서야 경기장튀김덕에 그래도 와본다 ...
2,https://blog.naver.com/iland2279/222888762836,가을은 변화가 아름다울 수 있다는 것을\n가르쳐주는 계절이라고 합니다.\n여름의 뜨...,2,가을은 변화가 아름다울 수 있다는 것을 가르쳐주는 계절이라고 합니다 여름의 뜨거웠던...
3,https://blog.naver.com/aceyun88/222888760839,"🚩정남진 장흥 여행 특산물, 귀족호도박물관,귀족호도를 아시나요?\n📌먼저 #정남진장...",3,정남진 장흥 여행 특산물 귀족호도박물관 귀족호도를 아시나요 먼저 정남진장흥 귀족호도...
4,https://blog.naver.com/numain64/222888760244,우린 보성녹차를 왔다...\n드라마촬영지를 포기하고 여기를 왔다.\n이번에는 자연을...,4,우린 보성녹차를 왔다 드라마촬영지를 포기하고 여기를 왔다 이번에는 자연을 더 보고 ...
...,...,...,...,...
982,https://blog.naver.com/ka1186/222864429353,#서울생활사박물관 #서울옛북부법조단지 #구치감동 #구치감전시실 #동일로 #노원구 #...,982,서울생활사박물관 서울옛북부법조단지 구치감동 구치감전시실 동일로 노원구 서울특별시 노...
983,https://blog.naver.com/ge0307/222864429290,🏊‍♀️\n우리 가족 건강 지키미 우리동네 체육관 이곳~\n주 3회 이상 매주 찾는...,983,우리 가족 건강 지키미 우리동네 체육관 이곳 주 3회 이상 매주 찾는 이 곳 광 교...
984,https://blog.naver.com/grant2/222864428566,"멸종 위기 야생식물 2급으로 지정된 단양쑥부쟁이의 자생지, 여주 가볼만한곳 강천섬을...",984,멸종 위기 야생식물 2급으로 지정된 단양쑥부쟁이의 자생지 여주 가볼만한곳 강천섬을 ...
985,https://blog.naver.com/tear911/222864426193,와~~~ 너무 시원한 한잔!!\n맥주 한잔과 맛있는 피자 파스타.\n여기 화덕촌 생...,985,와 너무 시원한 한잔 맥주 한잔과 맛있는 피자 파스타 여기 화덕촌 생긴거 나만 몰랐...


In [5]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

  0%|          | 0/987 [00:00<?, ?it/s]

,url,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/plumtreelee-/222888764192,#2022정조대왕능행차공동재현 #정조대왕능행차\n2022 정조대왕 능행차 공동재현\...,0,2022정조대왕능행차공동재현 정조대왕능행차 2022 정조대왕 능행차 공동재현 Kin...,[(2022정조대왕능행차공동재현 정조대왕능행차 2022 정조대왕 능행차 공동재현 K...
1,https://blog.naver.com/kimjihye525/222888763332,연꽃이 필때 와보고싶었지만\n더위에 약한 나는 이제서야 ...\n경기장튀김덕에 그래...,1,연꽃이 필때 와보고싶었지만 더위에 약한 나는 이제서야 경기장튀김덕에 그래도 와본다 ...,[(연꽃이 필때 와보고싶었지만 더위에 약한 나는 이제서야 경기장튀김덕에 그래도 와본...
2,https://blog.naver.com/iland2279/222888762836,가을은 변화가 아름다울 수 있다는 것을\n가르쳐주는 계절이라고 합니다.\n여름의 뜨...,2,가을은 변화가 아름다울 수 있다는 것을 가르쳐주는 계절이라고 합니다 여름의 뜨거웠던...,"[(가을은 변화가 아름다울 수 있다는 것을 가르쳐주는 계절이라고 합니다, 0, 37..."
3,https://blog.naver.com/aceyun88/222888760839,"🚩정남진 장흥 여행 특산물, 귀족호도박물관,귀족호도를 아시나요?\n📌먼저 #정남진장...",3,정남진 장흥 여행 특산물 귀족호도박물관 귀족호도를 아시나요 먼저 정남진장흥 귀족호도...,"[(정남진 장흥 여행 특산물 귀족호도박물관 귀족호도를 아시나요, 0, 32, Non..."
4,https://blog.naver.com/numain64/222888760244,우린 보성녹차를 왔다...\n드라마촬영지를 포기하고 여기를 왔다.\n이번에는 자연을...,4,우린 보성녹차를 왔다 드라마촬영지를 포기하고 여기를 왔다 이번에는 자연을 더 보고 ...,"[(우린 보성녹차를 왔다 드라마촬영지를 포기하고 여기를 왔다, 0, 31, None..."
...,...,...,...,...,...
982,https://blog.naver.com/ka1186/222864429353,#서울생활사박물관 #서울옛북부법조단지 #구치감동 #구치감전시실 #동일로 #노원구 #...,982,서울생활사박물관 서울옛북부법조단지 구치감동 구치감전시실 동일로 노원구 서울특별시 노...,[(서울생활사박물관 서울옛북부법조단지 구치감동 구치감전시실 동일로 노원구 서울특별시...
983,https://blog.naver.com/ge0307/222864429290,🏊‍♀️\n우리 가족 건강 지키미 우리동네 체육관 이곳~\n주 3회 이상 매주 찾는...,983,우리 가족 건강 지키미 우리동네 체육관 이곳 주 3회 이상 매주 찾는 이 곳 광 교...,[(우리 가족 건강 지키미 우리동네 체육관 이곳 주 3회 이상 매주 찾는 이 곳 광...
984,https://blog.naver.com/grant2/222864428566,"멸종 위기 야생식물 2급으로 지정된 단양쑥부쟁이의 자생지, 여주 가볼만한곳 강천섬을...",984,멸종 위기 야생식물 2급으로 지정된 단양쑥부쟁이의 자생지 여주 가볼만한곳 강천섬을 ...,[(멸종 위기 야생식물 2급으로 지정된 단양쑥부쟁이의 자생지 여주 가볼만한곳 강천섬...
985,https://blog.naver.com/tear911/222864426193,와~~~ 너무 시원한 한잔!!\n맥주 한잔과 맛있는 피자 파스타.\n여기 화덕촌 생...,985,와 너무 시원한 한잔 맥주 한잔과 맛있는 피자 파스타 여기 화덕촌 생긴거 나만 몰랐...,[(와 너무 시원한 한잔 맥주 한잔과 맛있는 피자 파스타 여기 화덕촌 생긴거 나만 ...


In [6]:
# 데이터프레임을 1000개 단위로 나누기
chunk_size = 10
num_chunks = len(df) // chunk_size + 1  # 나머지가 있는 경우 추가적인 청크 필요

dfs = {}
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_df = df[start_idx:end_idx]
    dfs[f"df_{i+1}"] = chunk_df

# 생성된 데이터프레임들 확인
for key, chunk_df in dfs.items():
    print(key)

df_1
df_2
df_3
df_4
df_5
df_6
df_7
df_8
df_9
df_10
df_11
df_12
df_13
df_14
df_15
df_16
df_17
df_18
df_19
df_20
df_21
df_22
df_23
df_24
df_25
df_26
df_27
df_28
df_29
df_30
df_31
df_32
df_33
df_34
df_35
df_36
df_37
df_38
df_39
df_40
df_41
df_42
df_43
df_44
df_45
df_46
df_47
df_48
df_49
df_50
df_51
df_52
df_53
df_54
df_55
df_56
df_57
df_58
df_59
df_60
df_61
df_62
df_63
df_64
df_65
df_66
df_67
df_68
df_69
df_70
df_71
df_72
df_73
df_74
df_75
df_76
df_77
df_78
df_79
df_80
df_81
df_82
df_83
df_84
df_85
df_86
df_87
df_88
df_89
df_90
df_91
df_92
df_93
df_94
df_95
df_96
df_97
df_98
df_99


In [7]:
# ''' 분리된 문장 한 개를 한 row로 '''
# df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

# ''' 문장 데이터 프레임 저장 '''
# df2.to_csv("./나무문화/2022/나무문화_본문_9월_전처리.csv", index=False, encoding='UTF-8')

In [7]:
df2 = pd.read_csv("./나무문화/2022/나무문화_본문_9월_전처리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [8]:
dfs['df_1']['split_list'].index

RangeIndex(start=0, stop=10, step=1)

In [9]:
len(dfs)

99

In [10]:
for a in range(1,len(dfs)+1):
    for i in tqdm(dfs['df_%s'%a]['split_list'].index):
        for j in range(len(dfs['df_%s'%a]['split_list'][i])):
            split_str= {'split_str': dfs['df_%s'%a]['split_list'][i][j][0], 'org_idx': [i]}
            print(split_str['org_idx'])
            split_df= pd.DataFrame(split_str)
            df2 = pd.concat([df2,split_df])
            df2.reset_index(drop=True, inplace=True)
            display.clear_output(wait=True) # 출력지우기
        else:
            time.sleep(1)

df2.reset_index(drop=True, inplace=True)
df2

[220]


In [ ]:
df2.to_csv("./나무문화/2022/나무문화_본문_9월_전처리.csv", index=False, encoding='UTF-8')